# WIP: An Interactive Plotting Backend for `pyrolite`

In [ ]:
from pyrolite.data.Aitchison import load_kongite
from pyrolite.util.log import Handle
from pyrolite.util.plot.backend import Backend

logger = Handle("pyrolite", level="DEBUG")

## Interface Switching

In [ ]:
with Backend("plotly"):
    with Backend("matplotlib"):
        ...
    ...

## Some Example Data

In [ ]:
from pyrolite.util.synthetic import example_spider_data

logger.setLevel("INFO")

ree_df = example_spider_data(
    noise_level=0.05,
    size=100,
    start="DMM_WH2005",
    norm_to="Chondrite_PON",
    offsets={"Eu": 0.2},
).pyrochem.REE
df = load_kongite()
df.head()


## Plot Comparison

In [ ]:
df.iloc[:, 1:3].pyroplot.scatter()

In [ ]:
with Backend("plotly"):
    fig = df.iloc[:, 1:3].pyroplot.scatter()

fig


In [ ]:
df.iloc[:, :3].pyroplot.scatter(color="k")

In [ ]:
with Backend("plotly"):
    fig = df.iloc[:, 1:4].pyroplot.ternary(color="k")

fig
# fig.write_html("ternary.html")


In [ ]:
ree_df.pyroplot.spider(unity_line=True, color="0.5", alpha=0.5)

In [ ]:
lambda_labels = (
    ree_df.pyrochem.lambda_lnREE(anomalies=["Eu"])
    .apply(lambda x: x.apply(lambda y: "{}: {:2.2f}".format(x.name, y)))
    .agg("<br>".join, axis=1)
)

with Backend("plotly"):
    fig = ree_df.pyroplot.spider(
        unity_line=True, color="0.5", alpha=0.5, width=600, text=lambda_labels
    )
fig
# fig.write_html("spider.html")
